<a href="https://colab.research.google.com/github/dko1217/porousmedia/blob/main/Unet_Random_Sphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

## System Information

In [ ]:
!cat /proc/cpuinfo

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

## Load Libraries

In [ ]:
!pip install hdf5storage
from hdf5storage import loadmat

import numpy as np
from numpy.random import seed
from numpy.random import default_rng

from matplotlib import pyplot as plt
import os

import scipy
from scipy import io

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.compat.v1.keras.backend import set_session

from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model

from sklearn.model_selection import train_test_split

tf.random.set_seed(280675)
np.random.seed(280675)

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/DarcyUnet Unit Cell/

print('tensorflow version : {}'.format(tf.__version__))

# Functions

In [ ]:
def minmax_transform(x, x_min=0, x_range=1):
  x = ( x - x_min ) / x_range
  return x


def shift_augmentation(my_solid, my_vel, shift_range, vel_dir):

  my_aug_solid = np.ones_like(my_solid)
  my_aug_vel = np.ones_like(my_vel)

  shift_x = shift_range[0]
  shift_y = shift_range[1]

  if(shift_x < 0):
    my_aug_solid[:shift_x,:,:] = my_solid[-1*shift_x:,:,:]
    my_aug_solid[shift_x:,:,:] = np.flip(my_solid, axis=0 )[:-1*shift_x,:,:]

    my_aug_vel[:shift_x,:,:] = my_vel[-1*shift_x:,:,:]
    if(vel_dir[0] == 'x'):
      my_aug_vel[shift_x:,:,:] = np.flip(-1*my_vel, axis=0 )[:-1*shift_x,:,:]
    elif(vel_dir[0] == 'y'):
      my_aug_vel[shift_x:,:,:] = np.flip(my_vel, axis=0 )[:-1*shift_x,:,:]
    else:
      my_aug_vel[shift_x:,:,:] = np.flip(my_vel, axis=0 )[:-1*shift_x,:,:]

  elif(shift_x > 0):
    my_aug_solid[:shift_x,:,:] = np.flip(my_solid, axis=0 )[-1*shift_x:,:,:]
    my_aug_solid[shift_x:,:,:] = my_solid[:-1*shift_x,:,:]

    if(vel_dir[0] == 'x'):
      my_aug_vel[:shift_x,:,:] = np.flip(-1*my_vel, axis=0 )[-1*shift_x:,:,:]
    elif(vel_dir[0] == 'y'):
      my_aug_vel[:shift_x,:,:] = np.flip(my_vel, axis=0 )[-1*shift_x:,:,:]
    else:
      my_aug_vel[:shift_x,:,:] = np.flip(my_vel, axis=0 )[-1*shift_x:,:,:]
    my_aug_vel[shift_x:,:,:] = my_vel[:-1*shift_x,:,:]
  else:
    my_aug_solid = my_solid
    my_aug_vel = my_vel

  my_solid = my_aug_solid
  my_vel = my_aug_vel

  my_aug_solid = np.ones_like(my_solid)
  my_aug_vel = np.ones_like(my_vel)

  if(shift_y < 0):
    my_aug_solid[:,:shift_y,:] = my_solid[:,-1*shift_y:,:]
    my_aug_solid[:,shift_y:,:] = np.flip(my_solid, axis=1 )[:,:-1*shift_y,:]

    my_aug_vel[:,:shift_y,:] = my_vel[:,-1*shift_y:,:]
    if(vel_dir[0] == 'x'):
      my_aug_vel[:,shift_y:,:] = np.flip(my_vel, axis=1 )[:,:-1*shift_y,:]
    elif(vel_dir[0] == 'y'):
      my_aug_vel[:,shift_y:,:] = np.flip(-1*my_vel, axis=1 )[:,:-1*shift_y,:]
    else:
      my_aug_vel[:,shift_y:,:] = np.flip(my_vel, axis=1 )[:,:-1*shift_y,:]
      
  elif(shift_y > 0):
    my_aug_solid[:,:shift_y,:] = np.flip(my_solid, axis=1 )[:,-1*shift_y:,:]
    my_aug_solid[:,shift_y:,:] = my_solid[:,:-1*shift_y,:]

    if(vel_dir[0] == 'x'):
      my_aug_vel[:,:shift_y,:] = np.flip(my_vel, axis=1 )[:,-1*shift_y:,:]
    elif(vel_dir[0] == 'y'):
      my_aug_vel[:,:shift_y,:] = np.flip(-1*my_vel, axis=1 )[:,-1*shift_y:,:]
    else:
      my_aug_vel[:,:shift_y,:] = np.flip(my_vel, axis=1 )[:,-1*shift_y:,:]
    my_aug_vel[:,shift_y:,:] = my_vel[:,:-1*shift_y,:]
  else:
    my_aug_solid = my_solid
    my_aug_vel = my_vel

  return my_aug_solid, my_aug_vel


def flip_augmentation(my_solid, my_vel, vel_dir, axis):

  my_aug_solid = np.flip(my_solid, axis=axis)

  if(vel_dir[0] == 'x'):
    if(axis == 0):
      my_aug_vel = np.flip(-1*my_vel, axis=axis)
    else:
      my_aug_vel = np.flip(my_vel, axis=axis)
  elif(vel_dir[0] == 'y'):
    if(axis == 1):
      my_aug_vel = np.flip(-1*my_vel, axis=axis)
    else:
      my_aug_vel = np.flip(my_vel, axis=axis)
  else:
    my_aug_vel = np.flip(my_vel, axis=axis)

  return my_aug_solid, my_aug_vel

# Custom Loss

In [ ]:
def div_loss1(y_true, y_pred):

  scale = 1

  mse = tf.math.reduce_mean( tf.math.square(y_true - y_pred) )
  
  dVxdx_true = (y_true[:,2:,1:-1,1:-1,0] - y_true[:,:-2,1:-1,1:-1,0])/2
  dVydy_true = (y_true[:,1:-1,2:,1:-1,1] - y_true[:,1:-1,:-2,1:-1,1])/2
  dVzdz_true = (y_true[:,1:-1,1:-1,2:,2] - y_true[:,1:-1,1:-1,:-2,2])/2
  div_true = dVxdx_true + dVydy_true + dVzdz_true

  dVxdx_pred = (y_pred[:,2:,1:-1,1:-1,0] - y_pred[:,:-2,1:-1,1:-1,0])/2
  dVydy_pred = (y_pred[:,1:-1,2:,1:-1,1] - y_pred[:,1:-1,:-2,1:-1,1])/2
  dVzdz_pred = (y_pred[:,1:-1,1:-1,2:,2] - y_pred[:,1:-1,1:-1,:-2,2])/2
  div_pred = dVxdx_pred + dVydy_pred + dVzdz_pred

  div_loss = tf.math.reduce_mean( tf.math.abs(div_true - div_pred) )

  loss = mse + div_loss*scale
  
  return loss


def div_loss2(y_true, y_pred):

  scale = 3

  mse = tf.math.reduce_mean( tf.math.square(y_true - y_pred) )
  
  dVxdx_pred = (y_pred[:,2:,1:-1,1:-1,0] - y_pred[:,:-2,1:-1,1:-1,0])/2
  dVydy_pred = (y_pred[:,1:-1,2:,1:-1,1] - y_pred[:,1:-1,:-2,1:-1,1])/2
  dVzdz_pred = (y_pred[:,1:-1,1:-1,2:,2] - y_pred[:,1:-1,1:-1,:-2,2])/2
  div_pred = dVxdx_pred + dVydy_pred + dVzdz_pred

  div_loss = tf.math.reduce_mean( tf.math.abs(div_pred) )

  loss = mse + div_loss*scale
  
  return loss

# PolySphere Data

In [ ]:
dir_data = 'Data/PolySphere'

data_size = 120

"""
X : ('x')
Y : ('y')
Z : ('z')
"""
#velocity_dir = ('x')
#velocity_dir = ('y')
#velocity_dir = ('z')
velocity_dir = ('x', 'y', 'z')

domainRange = [1,2,3,4,5,6,7,8,9]

channel_size = len(velocity_dir)

uc_solid = np.zeros( (1,data_size, data_size, data_size) )
uc_vel = np.zeros( (1,data_size, data_size, data_size, len(velocity_dir)) )

for i in range(len(domainRange)):
  
    uc_solid_load = loadmat( '{}/PolySphere_domain{}_deci.mat'.format(dir_data, domainRange[i]) )['solid'].astype('int')
    uc_solid_load = uc_solid_load < 1
    uc_solid = np.append( uc_solid, np.expand_dims(uc_solid_load, axis=0), axis=0 )
    del uc_solid_load

    uc_vel_load = np.zeros( (1, data_size, data_size, data_size, channel_size) )
    for j in range(channel_size):
      if(velocity_dir[j] == 'z'):
        uc_vel_load[0,:,:,:,j] = loadmat( '{}/PolySphere_domain{}_vfield{}.mat'.format(dir_data, domainRange[i], '') )['vfield'].astype('float32')
      else:
        uc_vel_load[0,:,:,:,j] = loadmat( '{}/PolySphere_domain{}_vfield{}.mat'.format(dir_data, domainRange[i], velocity_dir[j]) )['vfield'].astype('float32')
    uc_vel = np.append( uc_vel, uc_vel_load, axis=0 )
    del uc_vel_load


uc_solid = uc_solid[1:,:,:,:]
uc_vel = uc_vel[1:,:,:,:,:]

print('Image size : {}\nNumber of Data : {}  {}'.format(data_size, uc_solid.shape, uc_vel.shape))

In [ ]:
from scipy.ndimage import distance_transform_edt
from scipy.ndimage import distance_transform
uc_solid_edt = distance_transform_edt(uc_solid)

In [ ]:
""" Normalization
"""
res = 20e-6

x_min = 0
x_max = 8

uc_vel_norm = uc_vel/(res**2)*0.333/9270

print( '\nMean velocity : {}'.format( uc_vel_norm.mean() ) )
print( '\nMin velocity : {}'.format( uc_vel_norm.min() ) )
print( '\nMax velocity : {}'.format( uc_vel_norm.max() ) )

print('\nX_min : {} and X_max : {}'.format(x_min, x_max))

uc_vel_minmax = minmax_transform(uc_vel_norm, x_min=x_min, x_range=x_max-x_min)

print( '\nMean velocity : {}'.format( uc_vel_minmax.mean() ) )
print( '\nMin velocity : {}'.format( uc_vel_minmax.min() ) )
print( '\nMax velocity : {}'.format( uc_vel_minmax.max() ) )

In [ ]:
augGen_seed = 10

aug_iter = 15

shift_range = 2
flip_range = 5

aug_uc_solid = np.zeros( (1, data_size, data_size, data_size) )
aug_uc_vel = np.zeros( (1, data_size, data_size, data_size, channel_size) )

""" Shift and Flip augmentation
"""

shift = data_size//shift_range
rnd_num_gen = default_rng(augGen_seed)
rnd_num = rnd_num_gen.integers(low=-shift, high=shift, size=(uc_solid.shape[0],aug_iter,2), endpoint=True)
rnd_num2 = rnd_num_gen.integers(low=0, high=10, size=(uc_solid.shape[0],aug_iter,2))

for i in range(uc_solid.shape[0]):

  my_solid = uc_solid[i,:,:,:]
  my_vel = uc_vel_minmax[i,:,:,:,:]

  for j in range(aug_iter): 
    if(channel_size <= 1):

      my_aug_solid, my_aug_vel = shift_augmentation(my_solid, my_vel[:,:,:,0], rnd_num[i,j,:], velocity_dir)
      if(rnd_num2[i,j,0] >= flip_range):
        my_aug_solid, my_aug_vel = flip_augmentation(my_aug_solid, my_aug_vel, velocity_dir, 0)
      if(rnd_num2[i,j,1] >= flip_range):
        my_aug_solid, my_aug_vel = flip_augmentation(my_aug_solid, my_aug_vel, velocity_dir, 1)

      aug_uc_solid = np.append(aug_uc_solid, np.expand_dims(my_aug_solid,axis=0), axis=0)
      aug_uc_vel = np.append(aug_uc_vel, np.expand_dims( np.expand_dims(my_aug_vel,axis=0), axis=-1 ), axis=0)

    elif(channel_size == 3):

      my_aug_solid, my_aug_vel_x = shift_augmentation(my_solid, my_vel[:,:,:,0], rnd_num[i,j,:], ('x'))
      my_aug_solid, my_aug_vel_y = shift_augmentation(my_solid, my_vel[:,:,:,1], rnd_num[i,j,:], ('y'))
      my_aug_solid, my_aug_vel_z = shift_augmentation(my_solid, my_vel[:,:,:,2], rnd_num[i,j,:], ('z'))

      if(rnd_num2[i,j,0] >= flip_range):
        my_aug_solid, my_aug_vel_x = flip_augmentation(my_aug_solid, my_aug_vel_x, ('x'), 0)
        my_aug_solid, my_aug_vel_y = flip_augmentation(my_aug_solid, my_aug_vel_y, ('y'), 0)
        my_aug_solid, my_aug_vel_z = flip_augmentation(my_aug_solid, my_aug_vel_z, ('z'), 0)
      if(rnd_num2[i,j,1] >= flip_range):
        my_aug_solid, my_aug_vel_x = flip_augmentation(my_aug_solid, my_aug_vel_x, ('x'), 1)
        my_aug_solid, my_aug_vel_y = flip_augmentation(my_aug_solid, my_aug_vel_y, ('y'), 1)
        my_aug_solid, my_aug_vel_z = flip_augmentation(my_aug_solid, my_aug_vel_z, ('z'), 1)

      my_aug_vel = np.append( np.append(np.expand_dims(my_aug_vel_x,axis=-1), np.expand_dims(my_aug_vel_y,axis=-1), axis=-1), np.expand_dims(my_aug_vel_z,axis=-1), axis=-1 )

      aug_uc_solid = np.append(aug_uc_solid, np.expand_dims(my_aug_solid,axis=0), axis=0)
      aug_uc_vel = np.append(aug_uc_vel, np.expand_dims(my_aug_vel,axis=0), axis=0)


aug_uc_solid = aug_uc_solid[1:,:,:,:]
aug_uc_vel = aug_uc_vel[1:,:,:,:,:]

total_number = aug_uc_solid.shape[0]

val_perc = 0.1
test_perc = 0.1

train_index, val_test_index = train_test_split(np.arange(total_number), test_size = val_perc + test_perc, random_state = augGen_seed)
val_index, test_index = train_test_split(val_test_index, test_size = test_perc/(val_perc+test_perc), random_state = augGen_seed)

print( 'Number of training samples : {}\nNumber of validation samples : {}\nNumber of test samples : {}\n'.format(len(train_index),len(val_index),len(test_index)) )

train_data_solid = np.expand_dims(aug_uc_solid[train_index], axis=-1)
val_data_solid = np.expand_dims(aug_uc_solid[val_index], axis=-1)
test_data_solid = np.expand_dims(aug_uc_solid[test_index], axis=-1)

train_data_vel = aug_uc_vel[train_index]
val_data_vel = aug_uc_vel[val_index]
test_data_vel = aug_uc_vel[test_index]

print(train_data_solid.shape)
print(val_data_solid.shape)
print(test_data_solid.shape)

print(train_data_vel.shape)
print(val_data_vel.shape)
print(test_data_vel.shape)

# Data Check

## Mass loss check

In [ ]:
uc_solid_temp = uc_solid[0,:,:,:]
uc_solid_temp = np.expand_dims(uc_solid_temp, axis=(-1))

uc_vel_temp = uc_vel_minmax[0,:,:,:,:]

dVxdx = (uc_vel_temp[2:,1:-1,1:-1,0] - uc_vel_temp[:-2,1:-1,1:-1,0])/2
dVydy = (uc_vel_temp[1:-1,2:,1:-1,1] - uc_vel_temp[1:-1,:-2,1:-1,1])/2
dVzdz = (uc_vel_temp[1:-1,1:-1,2:,2] - uc_vel_temp[1:-1,1:-1,:-2,2])/2
div = dVxdx + dVydy + dVzdz

In [ ]:
plt.imshow(uc_solid_temp[:,:,59,0])
plt.show()

plt.imshow(uc_vel_temp[:,:,59,2])
plt.colorbar()
plt.show()

plt.imshow(div[:,:,59], vmin=-0.3, vmax=0.3)
plt.colorbar()
plt.show()

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.3,0.3,100))
plt.show()

print(np.mean( np.abs(div_flat) ) )

In [ ]:
xversion = 'X1-3'
x_model_name = 'UnetRS_Modelv{}'.format(xversion)
x_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(x_model_name, x_model_name ) )

yversion = 'Y1-1'
y_model_name = 'UnetRS_Modelv{}'.format(yversion)
y_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(y_model_name, y_model_name ) )

zversion = 'Z1-4'
z_model_name = 'UnetRS_Modelv{}'.format(zversion)
z_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(z_model_name, z_model_name ) )

vx_test_pred = np.float32( x_model.predict( x=[np.expand_dims(uc_solid_temp,axis=0)] ) )/2
vy_test_pred = np.float32( y_model.predict( x=[np.expand_dims(uc_solid_temp,axis=0)] ) )/2
vz_test_pred = np.float32( z_model.predict( x=[np.expand_dims(uc_solid_temp,axis=0)] ) )

vpred_test_temp = np.append(vx_test_pred, np.append(vy_test_pred, vz_test_pred,axis=-1),axis=-1 )
print(vpred_test_temp.shape)

In [ ]:
temp_vel = np.multiply(vpred_test_temp[0,:,:,:,:],uc_solid_temp[:,:,:,:])
#temp_vel = vpred_test_temp[0,:,:,:,:]

dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
div = dVxdx + dVydy + dVzdz

print(uc_solid_temp.shape)
plt.imshow(uc_solid_temp[:,:,59,0])
plt.show()

plt.imshow(temp_vel[:,:,59,2])
plt.colorbar()
plt.show()

plt.imshow(div[:,:,59],vmin=-0.3,vmax=0.3)
plt.colorbar()
plt.show()

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.3,0.3,100))
plt.show()

print(np.mean( np.abs(div_flat) ) )

In [ ]:
temp_div_loss = div_loss1(np.expand_dims(uc_vel_temp,axis=0), np.expand_dims(temp_vel,axis=0))

print(temp_div_loss)

## Augmentation test

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
temp = np.zeros( (1,20,20,1) )
temp[0,15:,:,0] = 10

plt.imshow(temp[0,:,:,0])
plt.show()

augGenTemp = ImageDataGenerator(#rotation_range=45, 
                              #width_shift_range=(data_size//4, 
                              height_shift_range=(5,5),
                              fill_mode='reflect')

iterator_temp = augGenTemp.flow(temp)


temp2 = iterator_temp.next()
print(temp2.shape)

plt.imshow(temp2[0,:,:,0])
plt.show()


In [ ]:
temp = np.zeros( (20,20,20) )
temp[13:,13:,:] = 10

plt.imshow(temp[:,:,9])
plt.show()

rnd_num_gen = default_rng(10)

# 1: height (x) 
# 2: width (y)
#rnd_num = rnd_num_gen.integers(low=-shift_range, high=shift_range, size=2, endpoint=True)
rnd_num = (-10,-10)

my_solid = temp
my_aug_solid = np.ones_like(temp)

if(rnd_num[0] < 0):
  my_aug_solid[:rnd_num[0],:,:] = my_solid[-1*rnd_num[0]:,:,:]
  my_aug_solid[rnd_num[0]:,:,:] = np.flip(my_solid, axis=0 )[:-1*rnd_num[0],:,:]
else:
  my_aug_solid[:rnd_num[0],:,:] = np.flip(my_solid, axis=0 )[-1*rnd_num[0]:,:,:]
  my_aug_solid[rnd_num[0]:,:,:] = my_solid[:-1*rnd_num[0],:,:]

my_solid = my_aug_solid
my_aug_solid = np.ones_like(temp)

if(rnd_num[1] < 0):
  my_aug_solid[:,:rnd_num[1],:] = my_solid[:,-1*rnd_num[1]:,:]
  my_aug_solid[:,rnd_num[1]:,:] = np.flip(my_solid, axis=1 )[:,:-1*rnd_num[1],:]
else:
  my_aug_solid[:,:rnd_num[1],:] = np.flip(my_solid, axis=1 )[:,-1*rnd_num[1]:,:]
  my_aug_solid[:,rnd_num[1]:,:] = my_solid[:,:-1*rnd_num[1],:]

plt.imshow(my_aug_solid[:,:,9])
plt.show()

In [ ]:
plt.imshow(my_solid[:,:,9])
plt.show()

rnd_num = (10,10)

if(rnd_num[1] < 0):
  my_aug_solid[:,:rnd_num[1],:] = my_solid[:,-1*rnd_num[1]:,:]
  my_aug_solid[:,rnd_num[1]:,:] = np.flip(my_solid, axis=1 )[:,:-1*rnd_num[1],:]
else:
  my_aug_solid[:,:rnd_num[1],:] = np.flip(my_solid, axis=1 )[:,-1*rnd_num[1]:,:]
  my_aug_solid[:,rnd_num[1]:,:] = my_solid[:,:-1*rnd_num[1],:]

plt.imshow(my_aug_solid[:,:,9])
plt.show()

In [ ]:
temp = np.array([1,2,3,4,5,6,67,7,8,9])

print(temp[:-3])

In [ ]:
#temp = np.zeros( (20,20,20) )
#temp[13:,13:,:] = 10

temp = uc_solid[0,:,:,:]
temp_v = uc_vel[0,:,:,:,0]

plt.imshow(temp[:,:,9])
plt.show()

plt.imshow(temp_v[:,:,9])
plt.show()

my_aug_solid = np.flip(temp, axis=0)
my_aug_vel = np.flip(-1*temp_v, axis=0)

plt.imshow(my_aug_solid[:,:,9])
plt.show()

plt.imshow(my_aug_vel[:,:,9])
plt.show()

## Save training data

In [ ]:
data_num = 1

np.save('Hoffman Cluster/train data/train_solid_randomsphere_data{}'.format(data_num), train_data_solid)
np.save('Hoffman Cluster/train data/train_vel{}_randomsphere_data{}'.format(velocity_dir, data_num), train_data_vel)

np.save('Hoffman Cluster/train data/val_solid_randomsphere_data{}'.format(data_num), val_data_solid)
np.save('Hoffman Cluster/train data/val_vel{}_randomsphere_data{}'.format(velocity_dir, data_num), val_data_vel)

np.save('Hoffman Cluster/train data/test_solid_randomsphere_data{}'.format(data_num), test_data_solid)
np.save('Hoffman Cluster/train data/test_vel{}_randomsphere_data{}'.format(velocity_dir, data_num), test_data_vel)

In [ ]:
velocity_dir = 'x'

augGen_seed = 10
aug_iter = 15

train_data_solid = np.load('Hoffman Cluster/train data/train_solid_randomsphere_augGenSeed_{}_AugIter_{}.npy'.format(augGen_seed, aug_iter))
train_data_vel = np.load('Hoffman Cluster/train data/train_vel{}_randomsphere_augGenSeed_{}_AugIter_{}.npy'.format(velocity_dir, augGen_seed, aug_iter))

val_data_solid = np.load('Hoffman Cluster/train data/val_solid_randomsphere_augGenSeed_{}_AugIter_{}.npy'.format(augGen_seed, aug_iter))
val_data_vel = np.load('Hoffman Cluster/train data/val_vel{}_randomsphere_augGenSeed_{}_AugIter_{}.npy'.format(velocity_dir, augGen_seed, aug_iter))

test_data_solid = np.load('Hoffman Cluster/train data/test_solid_randomsphere_augGenSeed_{}_AugIter_{}.npy'.format(augGen_seed, aug_iter))
test_data_vel = np.load('Hoffman Cluster/train data/test_vel{}_randomsphere_augGenSeed_{}_AugIter_{}.npy'.format(velocity_dir, augGen_seed, aug_iter))

print(train_data_solid.shape)
print(val_data_solid.shape)
print(test_data_solid.shape)

print(train_data_vel.shape)
print(val_data_vel.shape)
print(test_data_vel.shape)

## Data Check

In [ ]:
vrange = np.linspace(0,10,100)
vel_norm_nvoxel = np.zeros(vrange.shape[0]-1)

test_data_vel_flat = test_data_vel.flatten()

for i in range(vel_norm_nvoxel.shape[0]):

  vel_norm_nvoxel[i] = np.sum( test_data_vel_flat[ (test_data_vel_flat >= vrange[i]) & (test_data_vel_flat < vrange[i+1]) ] )

plt.bar(vrange[:-1], vel_norm_nvoxel)
plt.xlabel('V')
plt.ylabel('Number of voxels')
plt.show()

In [ ]:
# Voxel histogram of minmax velocity

percentile = np.linspace(-10,10,200)
vel_nvoxel = np.zeros(percentile.shape[0]-1)

for i in range(vel_nvoxel.shape[0]):

  vel_masked = np.ma.masked_outside(uc_vel_minmax[2,:,:,:], percentile[i], percentile[i+1])
  vel_filled = np.ma.filled(vel_masked, fill_value=0)
  vel_nvoxel[i] = vel_filled[vel_filled != 0].shape[0]

plt.bar(percentile[:-1], vel_nvoxel)
plt.xlabel('V')
plt.ylabel('Number of voxels')
plt.show()

In [ ]:
raw_data_perm = np.zeros( (5, uc_solid.shape[0]) )
norm_data_perm = np.zeros( (5, uc_solid.shape[0]) )

x_max = 50
res = 20e-6

for i in range(raw_data_perm.shape[1]):
  norm_data_perm[0,i] = uc_vel_minmax[i,:,:,:].mean()
  raw_data_perm[0,i] = norm_data_perm[0,i]*x_max*(res**2)
  raw_data_perm[1,i] = uc_vel_minmax[i,:60,:60,:].mean()*x_max*(res**2)
  raw_data_perm[2,i] = uc_vel_minmax[i,:60,60:,:].mean()*x_max*(res**2)
  raw_data_perm[3,i] = uc_vel_minmax[i,60:,:60,:].mean()*x_max*(res**2)
  raw_data_perm[4,i] = uc_vel_minmax[i,60:,60:,:].mean()*x_max*(res**2)


plt.hist(raw_data_perm[0,:], bins=20)
plt.title('Permeability')
plt.show()
plt.hist(norm_data_perm[0,:], bins=20)
plt.title('Normalized Permeability')
plt.xlim([0.14, 0.24])
plt.show()

plt.subplot(221)
plt.hist(raw_data_perm[1,:], bins=10)

plt.subplot(222)
plt.hist(raw_data_perm[2,:], bins=10)

plt.subplot(223)
plt.hist(raw_data_perm[3,:], bins=10)

plt.subplot(224)
plt.hist(raw_data_perm[4,:], bins=10)
plt.show()

In [ ]:
train_data_perm = np.zeros( (5, train_data_solid.shape[0]) )
train_data_norm = np.zeros( (5, train_data_solid.shape[0]) )

for i in range(train_data_perm.shape[1]):
  train_data_norm[0,i] = train_data_vel[i,:,:,:,0].mean()
  train_data_perm[0,i] = train_data_norm[0,i]*x_max*(res**2)
  train_data_perm[1,i] = train_data_vel[i,:60,:60,:,0].mean()*x_max*(res**2)
  train_data_perm[2,i] = train_data_vel[i,:60,:60,:,0].mean()*x_max*(res**2)
  train_data_perm[3,i] = train_data_vel[i,:60,:60,:,0].mean()*x_max*(res**2)
  train_data_perm[4,i] = train_data_vel[i,:60,:60,:,0].mean()*x_max*(res**2)

plt.hist(train_data_perm[0,:], bins=10)
plt.title('Raw Permeability')
plt.show()

plt.hist(train_data_norm[0,:], bins=10)
plt.title('Normalized Permeability')
plt.show()

plt.subplot(221)
plt.hist(train_data_perm[1,:], bins=10)
plt.title('I')

plt.subplot(222)
plt.hist(train_data_perm[2,:], bins=10)
plt.title('II')

plt.subplot(223)
plt.hist(train_data_perm[3,:], bins=10)
plt.title('III')

plt.subplot(224)
plt.hist(train_data_perm[4,:], bins=10)
plt.title('IV')

plt.show()


## Illustrations

In [ ]:
# Raw data
sample_number = 2

slice = np.array( [0, 59, -1] )
fig_title = ['Inlet', 'Midpoint', 'Outlet']

fig, axs = plt.subplots( nrows=2, ncols=3,figsize=(10,10) )

vel_magnitude = 1e-4

for j in np.array( [0, 1, 2] ):
  im=axs[0,j].imshow(uc_solid[sample_number,:,:,slice[j]], clim=(0,1), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[0,j],fraction=0.05)
  axs[0,j].axis('off')
  axs[0,j].set_title('%s Solid' % (fig_title[j]))

  im=axs[1,j].imshow(uc_vel[sample_number,:,:,slice[j],0], clim=(0,vel_magnitude), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[1,j],fraction=0.05)
  axs[1,j].axis('off')
  axs[1,j].set_title('%s Velocity' % (fig_title[j]))

In [ ]:
# Norm data
sample_number = 0

slice = np.array( [0, 59, -1] )
fig_title = ['Inlet', 'Midpoint', 'Outlet']

fig, axs = plt.subplots( nrows=2, ncols=3,figsize=(10,10) )

for j in np.array( [0, 1, 2] ):
  im=axs[0,j].imshow(uc_solid[sample_number,:,:,slice[j]], clim=(0,1), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[0,j],fraction=0.05)
  axs[0,j].axis('off')
  axs[0,j].set_title('%s Solid' % (fig_title[j]))

  im=axs[1,j].imshow(uc_vel_norm[sample_number,:,:,slice[j]], clim=(0,40), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[1,j],fraction=0.05)
  axs[1,j].axis('off')
  axs[1,j].set_title('%s Velocity' % (fig_title[j]))

In [ ]:
# Minmax data
sample_number = 5

slice = np.array( [0, 59, -1] )
fig_title = ['Inlet', 'Midpoint', 'Outlet']

fig, axs = plt.subplots( nrows=2, ncols=3,figsize=(10,10) )

vel_magnitude = 10

for j in np.array( [0, 1, 2] ):
  im=axs[0,j].imshow(uc_solid[sample_number,:,:,slice[j]], clim=(0,1), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[0,j],fraction=0.05)
  axs[0,j].axis('off')
  axs[0,j].set_title('%s Solid' % (fig_title[j]))

  im=axs[1,j].imshow(uc_vel_minmax[sample_number,:,:,slice[j],0], clim=(0,vel_magnitude), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[1,j],fraction=0.05)
  axs[1,j].axis('off')
  axs[1,j].set_title('%s Velocity' % (fig_title[j]))

In [ ]:
# Data check - train data
sample_number = 2

slice = np.array( [0, 59, -1] )
fig_title = ['Inlet', 'Midpoint', 'Outlet']

fig, axs = plt.subplots( nrows=2, ncols=3,figsize=(10,10) )

for j in np.array( [0, 1, 2] ):
  im=axs[0,j].imshow(train_data_solid[sample_number,:,:,slice[j],0], clim=(0,1), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[0,j],fraction=0.05)
  axs[0,j].axis('off')
  axs[0,j].set_title('%s Solid' % (fig_title[j]))

  im=axs[1,j].imshow(train_data_vel[sample_number,:,:,slice[j],0], clim=(-2,2), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[1,j],fraction=0.05)
  axs[1,j].axis('off')
  axs[1,j].set_title('%s Velocity' % (fig_title[j]))

# Darcy Unet Training

## Unet Model

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, Conv3D, Conv3DTranspose, BatchNormalization, Activation, Concatenate, Dropout, Multiply

from numpy import floor, ceil

def encoder_block(inputs, strides, filter_num, filter_size, activation, momentum, rate):

  path = Conv3D(filter_num, filter_size, padding='same', strides=strides)(inputs)
  path = BatchNormalization(momentum=momentum)(path)
  path = Activation(activation=activation)(path)
  path = Dropout(rate)(path)

  return path


def decoder_block(inputs, strides, filter_num, filter_size, activation, momentum, rate):

  path = Conv3DTranspose(filter_num, filter_size, padding='same', strides=strides)(inputs)
  path = BatchNormalization(momentum=momentum)(path)
  path = Activation(activation=activation)(path)
  path = Dropout(rate)(path)

  return path


def UnetV1(input_shape, filter_num = 5, filter_size = 3, activation = 'selu', momentum = 0.99, rate = 0.2):

  inputs = Input(shape = input_shape)

  skip_connection = []

  for i in range(8):
    
    if(i <= 0):
      path_encoder = encoder_block(inputs, 1, filter_num*(2**floor(i/2)), filter_size, activation, momentum, rate)
    else:

      if(i % 2 == 1):
        path_encoder = encoder_block(path_encoder, 1, filter_num*(2**floor(i/2)), filter_size, activation, momentum, rate)
        skip_connection.append(path_encoder)
      else:
        path_encoder = encoder_block(path_encoder, 2, filter_num*(2**floor(i/2)), filter_size, activation, momentum, rate)

  for i in reversed(range(6)):

    if(i >= 5):
      path_decoder = decoder_block(path_encoder, 2, filter_num*(2**floor(i/2)), filter_size, activation, momentum, rate)
      path_decoder = Concatenate()([ path_decoder, skip_connection[int(floor(i/2))] ])
    else:

      if(i % 2 == 0):
        path_decoder = decoder_block(path_decoder, 1, filter_num*(2**floor(i/2)), filter_size, activation, momentum, rate)
      else:
        path_decoder = decoder_block(path_decoder, 2, filter_num*(2**floor(i/2)), filter_size, activation, momentum, rate)
        path_decoder = Concatenate()([ path_decoder, skip_connection[int(floor(i/2))] ])


  path = Conv3D(filter_num, filter_size, padding='same')(path_decoder)
  path = Conv3D(1, 1, padding='same')(path)

  #for version 2
  #path = Multiply()([inputs, path])

  return Model(inputs=inputs, outputs=path)


def UnetV2(input_shape, x_model, y_model, z_model, filter_num = 5, filter_size = 3, activation = 'selu', momentum = 0.99, rate = 0.2):

  v_scale = 1
  
  inputs = Input(shape = input_shape)

  x_input = tf.math.multiply( x_model(inputs, training=False), 0.5*v_scale )
  y_input = tf.math.multiply( y_model(inputs, training=False), 0.5*v_scale )
  z_input = tf.math.multiply( z_model(inputs, training=False), 1*v_scale )

  path = Concatenate()( [x_input, y_input, z_input] )

  skip_connection = []

  for i in range(8):
  
    if(i <= 0):
      path_encoder = encoder_block(path, 1, filter_num, filter_size, activation, momentum, rate)
    else:

      if(i % 2 == 1):
        path_encoder = encoder_block(path_encoder, 1, filter_num, filter_size, activation, momentum, rate)
        skip_connection.append(path_encoder)
      else:
        path_encoder = encoder_block(path_encoder, 2, filter_num, filter_size, activation, momentum, rate)

  for i in reversed(range(6)):

    if(i >= 5):
      path_decoder = decoder_block(path_encoder, 2, filter_num, filter_size, activation, momentum, rate)
      path_decoder = Concatenate()([ path_decoder, skip_connection[int(floor(i/2))] ])
    else:

      if(i % 2 == 0):
        path_decoder = decoder_block(path_decoder, 1, filter_num, filter_size, activation, momentum, rate)
      else:
        path_decoder = decoder_block(path_decoder, 2, filter_num, filter_size, activation, momentum, rate)
        path_decoder = Concatenate()([ path_decoder, skip_connection[int(floor(i/2))] ])


  path = Conv3D(filter_num, filter_size, padding='same')(path_decoder)
  path = Conv3D(3, 1, padding='same')(path)

  return Model(inputs=inputs, outputs=path)

## Model Training

In [ ]:
version = 'Z1-4-Seed3'

model_name = 'UnetRS_Modelv{}'.format(version)

dir_save   = 'RandomSphere Model/{}'.format(model_name)
try:
  os.mkdir(dir_save)
except OSError as error:
  print(error)

In [ ]:
filter_size = 4
num_filters = 10

learning_rate = 0.0006
batch_size    = 4

momentum = 0.9
rate = 0.1

epochs        = 1000
patience_training = 50

metrics=['MAE', 'MSE']
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

model = UnetV1( input_shape = (data_size, data_size, data_size, 1), filter_num = num_filters, filter_size = filter_size, activation = 'selu', momentum = momentum, rate = rate)

#model.summary(line_length = 250)

model.compile( loss = tf.keras.losses.mean_squared_error, optimizer=optimizer, metrics=metrics[:] )

nan_terminate = tf.keras.callbacks.TerminateOnNaN()
early_stop    = tf.keras.callbacks.EarlyStopping(monitor ='val_loss', min_delta = 0,
                                              patience = patience_training, 
                                              verbose = True, mode = 'auto', baseline = None)

csv_logger = tf.keras.callbacks.CSVLogger("{}/training_log_{}.csv".format(dir_save,model_name))

checkpoint = ModelCheckpoint('{}/{}.ckpt'.format(dir_save,model_name), 
                             monitor = 'val_loss', 
                             verbose = 1, 
                             save_best_only = True, 
                             mode = 'min', save_weights_only = False)

callbacks_list = [nan_terminate,
                  early_stop,
                  checkpoint,
                  csv_logger]

In [ ]:
from keras.utils.vis_utils import plot_model

imgtype = ('LR','TB')
for i in imgtype:
  plot_model(model, to_file='RandomSphere Model/Model Image/{}_{}.jpg'.format(model_name, i), rankdir = i, show_layer_names = False)

In [ ]:
from timeit import default_timer as timer
from datetime import timedelta

start = timer()

model.fit( x = train_data_solid, y = train_data_vel, 
           epochs = epochs, batch_size = batch_size,
           validation_data = (val_data_solid, val_data_vel),
           validation_freq = 1,
           verbose = 1,
           callbacks = callbacks_list)

end = timer()
print('Elapsed time for training : {}'.format(timedelta(seconds=end-start)))   

## XYZ Model Training

In [ ]:
version = 'XYZ1-8-Seed3'

model_name = 'UnetRSXYZ_Modelv{}'.format(version)

dir_save   = 'RandomSphere XYZ Model/{}'.format(model_name)
try:
  os.mkdir(dir_save)
except OSError as error:
  print(error)

In [ ]:
filter_size = 3
num_filters = 9

learning_rate = 0.001
batch_size    = 4

momentum = 0.99
rate = 0.0001

epochs        = 1000
patience_training = 50

metrics=['MAE', 'MSE']
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)


version_x = 'X1-3'
x_model_name = 'UnetRS_Modelv{}'.format(version_x)
x_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(x_model_name, x_model_name ) )
x_model._name = 'xmodel'
x_model.trainable = False

version_y = 'Y1-1'
y_model_name = 'UnetRS_Modelv{}'.format(version_y)
y_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(y_model_name, y_model_name ) )
y_model._name = 'ymodel'
y_model.trainable = False

version_z = 'Z1-4'
z_model_name = 'UnetRS_Modelv{}'.format(version_z)
z_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(z_model_name, z_model_name ) )
z_model._name = 'zmodel'
z_model.trainable = False

model = UnetV2( input_shape = (data_size, data_size, data_size, 1), x_model = x_model, y_model = y_model, z_model = z_model,
                filter_num = num_filters, filter_size = filter_size, activation = 'selu', momentum = momentum, rate = rate)

#model.summary(line_length = 250)

#model.compile( loss = tf.keras.losses.mean_squared_error, optimizer=optimizer, metrics=metrics[:] )
model.compile( loss = div_loss2, optimizer=optimizer, metrics=metrics[:])

nan_terminate = tf.keras.callbacks.TerminateOnNaN()
early_stop    = tf.keras.callbacks.EarlyStopping(monitor ='val_loss', min_delta = 0,
                                              patience = patience_training, 
                                              verbose = True, mode = 'auto', baseline = None)

csv_logger = tf.keras.callbacks.CSVLogger("{}/training_log_{}.csv".format(dir_save,model_name))

checkpoint = ModelCheckpoint('{}/{}.ckpt'.format(dir_save,model_name), 
                             monitor = 'val_loss', 
                             verbose = 1, 
                             save_best_only = True, 
                             mode = 'min', save_weights_only = False)

callbacks_list = [nan_terminate,
                  early_stop,
                  checkpoint,
                  csv_logger]

In [ ]:
from keras.utils.vis_utils import plot_model

imgtype = ('LR','TB')
for i in imgtype:
  plot_model(model, to_file='RandomSphere Model/Model Image/{}_{}.jpg'.format(model_name, i), rankdir = i, show_layer_names = False)

In [ ]:
from timeit import default_timer as timer
from datetime import timedelta

start = timer()

model.fit( x = train_data_solid, y = train_data_vel, 
           epochs = epochs, batch_size = batch_size,
           validation_data = (val_data_solid, val_data_vel),
           validation_freq = 1,
           verbose = 1,
           callbacks = callbacks_list)

end = timer()
print('Elapsed time for training : {}'.format(timedelta(seconds=end-start)))   

## Model Evaluation

In [ ]:
#version = 'Z1-4'
#eval_model_name = 'UnetRS_Modelv{}'.format(version)
#eval_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(eval_model_name, eval_model_name ) )

version = 'XYZ1-0'
eval_model_name = 'UnetRSXYZ_Modelv{}'.format(version)
eval_model = tf.keras.models.load_model( 'RandomSphere XYZ Model/{}/{}.ckpt'.format(eval_model_name, eval_model_name ) )
#eval_model = tf.keras.models.load_model( 'RandomSphere XYZ Model/{}/{}.ckpt'.format(eval_model_name, eval_model_name ), custom_objects = {'div_loss2': div_loss2} )

for i in range(test_data_solid.shape[0]):
  print('\nSample number : {}'.format(i+1))
  eval_model.evaluate(test_data_solid[i:i+1,:,:,:,:], test_data_vel[i:i+1,:,:,:,:])

vz_test_pred = np.float32( eval_model.predict( x=[test_data_solid] ) )

print(vz_test_pred.shape)

In [ ]:
# Overall Permeability

perm_true = test_data_vel[:,:,:,:,2].mean(axis=(1,2,3))
perm_pred = vz_test_pred[:,:,:,:,2].mean(axis=(1,2,3))

perm_error = abs( perm_true - perm_pred )/abs(perm_true)*100

print( 'Overall permeability error : {:.3f}\n'.format(perm_error.mean()) )

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(np.arange(perm_error.shape[0]), perm_error)
plt.title('Permeability Error')
plt.show()

"""
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(np.arange(perm_error.shape[0]),test_data_vel[:,:,:,:,0].mean(axis=(1,2,3)))
plt.title('Average velocity')
plt.show()
"""

print(perm_error)

In [ ]:
# Voxel-wise MAPE Absolute value velocity
v_max = 5
channel = 2

vel_true_flat = test_data_vel[:,:,:,:,channel].flatten()
vel_pred_flat = vz_test_pred[:,:,:,:,channel].flatten()

v_range = np.linspace(0.0001,v_max,100)

vel_true_flat_nonzero = vel_true_flat[ (vel_true_flat < 0) | (vel_true_flat > 0)]
vel_pred_flat_nonzero = vel_pred_flat[ (vel_true_flat < 0) | (vel_true_flat > 0)] 

vel_mape_flat = (vel_true_flat_nonzero - vel_pred_flat_nonzero)/vel_true_flat_nonzero*100

plt.plot(vel_true_flat_nonzero, vel_mape_flat,'bo',markersize=0.1,figure=plt.figure(figsize=[8,6]))
plt.ylim(-100,100)
plt.show()


vel_mape_flat_range = np.zeros_like(v_range)

for i in range(v_range.shape[0]-1):
  if(v_range[i] != 0):
    vel_mape_flat_vrange = np.abs( vel_mape_flat[ (np.abs(vel_true_flat_nonzero) > v_range[i]) & (np.abs(vel_true_flat_nonzero) < v_range[i+1]) ] )

  if(vel_mape_flat_vrange.shape[0] > 0):
    vel_mape_flat_range[i] = np.mean(vel_mape_flat_vrange)

plt.plot(v_range[:-1], vel_mape_flat_range[:-1], 'bo',markersize=4,figure=plt.figure(figsize=[8,6]))
plt.ylim(0,100)
plt.xlabel('velocity')
plt.ylabel('% error')
plt.show()


threshold = 9.17/x_max

vel_mape_flat_threshold = vel_mape_flat[ (vel_true_flat_nonzero <= (-1*threshold)) | (vel_true_flat_nonzero >= (1*threshold)) ]
print("% Error above threshold : {}".format(np.mean(vel_mape_flat_threshold[vel_mape_flat_threshold > 0])))

#print(v_range)
#print(vel_mape_flat_range)

In [ ]:
test_sample = 1
print( 'Maximum Sample Number : {}'.format(test_data_solid.shape[0] - 1) )

mySolid = test_data_solid[test_sample,:,:,:,0]
mySolid_mask = np.ma.masked_less(mySolid,1)

myVel_true = test_data_vel[test_sample,:,:,:,2]
myVel_true = np.ma.array( myVel_true, mask=np.ma.getmask(mySolid_mask) )

myVel_pred = vz_test_pred[test_sample,:,:,:,2]
myVel_pred = np.ma.array( myVel_pred, mask=np.ma.getmask(mySolid_mask) )

myVel_true = myVel_true*x_max
myVel_pred = myVel_pred*x_max

test_slice = np.array( [0, 59, -1] )
fig_title = ['Inlet', 'Midpoint', 'Outlet']

fig, axs = plt.subplots( nrows=3, ncols=3,figsize=(20,20) )

vel_range = (0,50)

for j in range(3):

  im=axs[j,0].imshow(mySolid[:,:,test_slice[j]], clim=(0,1), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[j,0],fraction=0.05)
  axs[j,0].axis('off')
  #axs[j,0].set_title('{} Solid'.format(fig_title[j]))

  im=axs[j,1].imshow(myVel_true[:,:,test_slice[j]], clim=vel_range, cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[j,1],fraction=0.05)
  axs[j,1].axis('off')
  #axs[j,1].set_title('{} Simulation'.format(fig_title[j]))

  im=axs[j,2].imshow(myVel_pred[:,:,test_slice[j]], clim=vel_range, cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[j,2],fraction=0.05)
  axs[j,2].axis('off')
  #axs[j,2].set_title('{} Prediction'.format(fig_title[j]))

In [ ]:
div = np.zeros( (vz_test_pred.shape[0],118,118,118) )
for i in range(vz_test_pred.shape[0]):

  temp_vel = vz_test_pred[i,:,:,:,:]

  dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
  dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
  dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
  div[i,:,:,:] = dVxdx + dVydy + dVzdz

print( np.mean( np.abs(div), axis=(1,2,3)) )

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.1,0.1,100))
plt.show()

print(np.mean( np.abs(div_flat) ))

In [ ]:
#temp_vel = vz_test_pred[0,:,:,:,:]*test_data_solid[0,:,:,:,:]
temp_vel = vz_test_pred[0,:,:,:,:]

dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
div = dVxdx + dVydy + dVzdz

plt.imshow(test_data_solid[0,:,:,59,0])
plt.show()

plt.imshow(temp_vel[:,:,59,2])
plt.colorbar()
plt.show()

plt.imshow(div[:,:,59])
plt.colorbar()
plt.show()

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.1,0.1,100))
plt.show()

print(np.mean( np.abs(div_flat) ))

In [ ]:
temp_vel = test_data_vel[0,:,:,:,:]

dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
div = dVxdx + dVydy + dVzdz

plt.imshow(test_data_solid[0,:,:,5,0])
plt.show()

plt.imshow(temp_vel[:,:,5,2])
plt.colorbar()
plt.show()

plt.imshow(div[:,:,5])
plt.colorbar()
plt.show()

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.1,0.1,100))
plt.show()

## Evaluate SiC Data

### Load SiC Data

In [ ]:
sic_dir_data = 'Data/SiC Data'
data_size = 120

denRange = [45, 65, 80]
domainRange = [1, 2]

# X : 'x'
# Y : 'y'
# Z : ''
#velocity_dir = ('x')
#velocity_dir = ('y')
#velocity_dir = ('z')
velocity_dir = ('x', 'y', 'z')

channel_size = len(velocity_dir)

sic_solid = np.zeros( (1,data_size,data_size,data_size,1) )
sic_vel = np.zeros( (1,data_size,data_size,data_size,channel_size) )

for i in range(len(denRange)):
  for j in range(len(domainRange)):

    sic_solid_load = loadmat( '{}/{}PPI_domain{}_deci.mat'.format(sic_dir_data, denRange[i], domainRange[j]) )['solid'].astype('int')
    sic_solid_load = sic_solid_load <= 0
    sic_solid = np.append( sic_solid, np.expand_dims(sic_solid_load, axis=(0,-1)), axis=0 )

    sic_vel_load = np.zeros( (1,data_size,data_size,data_size,channel_size))
    for k in range(channel_size):
      if(velocity_dir[k] == 'z'):
        sic_vel_load[0,:,:,:,k] = loadmat( '{}/{}PPI_domain{}_vfield{}.mat'.format(sic_dir_data, denRange[i], domainRange[j], '') )['vfield'].astype('float32')
      else:
        sic_vel_load[0,:,:,:,k] = loadmat( '{}/{}PPI_domain{}_vfield{}.mat'.format(sic_dir_data, denRange[i], domainRange[j], velocity_dir[k]) )['vfield'].astype('float32')

    sic_vel = np.append( sic_vel, sic_vel_load, axis=0 )

sic_solid = sic_solid[1:,:,:,:]
sic_vel = sic_vel[1:,:,:,:]

print('Image size : {}\nNumber of Data : {}  {}'.format(data_size, sic_solid.shape, sic_vel.shape))

In [ ]:
""" Normalization
"""

res = 26.708e-6
res2 = res/2

x_min = 0
x_max = 8

sic_vel_norm = sic_vel[:4,:,:,:,:]/(res**2)*0.333/9270
sic_vel_norm = np.append(sic_vel_norm,sic_vel[4:,:,:,:,:]/(res2**2)*0.333/9270,axis=0)

print( '\nMean velocity : {}'.format( sic_vel_norm.mean() ) )
print( '\nMin velocity : {}'.format( sic_vel_norm.min() ) )
print( '\nMax velocity : {}'.format( sic_vel_norm.max() ) )

sic_vel_minmax = minmax_transform(sic_vel_norm, x_min=x_min, x_range=x_max-x_min)

print( '\nMean velocity : {}'.format( sic_vel_minmax.mean() ) )
print( '\nMin velocity : {}'.format( sic_vel_minmax.min() ) )
print( '\nMax velocity : {}'.format( sic_vel_minmax.max() ) )

### SiC Data check

In [ ]:
data_num = 1

np.save('Data/SiC Data/sic_solid_{}'.format(data_num), sic_solid)

In [ ]:
vrange = np.linspace(-30,30,200)
vel_norm_nvoxel = np.zeros(vrange.shape[0]-1)

sic_vel_norm_flat = sic_vel_norm.flatten()

for i in range(vel_norm_nvoxel.shape[0]):

  vel_norm_nvoxel[i] = np.abs( np.sum( sic_vel_norm_flat[ (sic_vel_norm_flat >= vrange[i]) & (sic_vel_norm_flat < vrange[i+1]) ] ) )

plt.bar(vrange[:-1], vel_norm_nvoxel)
plt.xlabel('V')
plt.ylabel('Number of voxels')
plt.show()

print(np.sum(sic_vel_norm_flat[sic_vel_norm_flat >= 25]))

In [ ]:
percentile = np.linspace(-5,5,100)
vel_norm_nvoxel = np.zeros(percentile.shape[0]-1)

for i in range(vel_norm_nvoxel.shape[0]):

  vel_norm_masked = np.ma.masked_outside(sic_vel_minmax, percentile[i], percentile[i+1])
  vel_norm_filled = np.ma.filled(vel_norm_masked, fill_value=0)
  vel_norm_nvoxel[i] = vel_norm_filled[vel_norm_filled != 0].shape[0]

plt.bar(percentile[:-1], vel_norm_nvoxel)
plt.xlabel('V')
plt.ylabel('Number of voxels')
plt.show()

In [ ]:
raw_sic_perm = np.zeros( (5, sic_solid.shape[0]) )
norm_sic_perm = np.zeros( (5, sic_solid.shape[0]) )

x_max = 50
res = 26.708e-6

for i in range(raw_sic_perm.shape[1]):
  norm_sic_perm[0,i] = sic_vel_minmax[i,:,:,:].mean()
  raw_sic_perm[0,i] = norm_sic_perm[0,i]*x_max*(res**2)

plt.hist(raw_sic_perm[0,:], bins=20)
plt.title('Raw Permeability')
plt.show()
plt.hist(norm_sic_perm[0,:], bins=20)
plt.title('Normalized Permeability')
plt.xlim([0.1, 0.24])
plt.show()

In [ ]:
# Data check
sample_number = 5

slice = np.array( [0, 59, -1] )
fig_title = ['Inlet', 'Midpoint', 'Outlet']

v_max = 9

fig, axs = plt.subplots( nrows=2, ncols=3,figsize=(10,10) )

for j in np.array( [0, 1, 2] ):
  im=axs[0,j].imshow(sic_solid[sample_number,:,:,slice[j],0], clim=(0,1), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[0,j],fraction=0.05)
  axs[0,j].axis('off')
  axs[0,j].set_title('%s Solid' % (fig_title[j]))

  im=axs[1,j].imshow(sic_vel_minmax[sample_number,:,:,slice[j],0], clim=(0,v_max), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[1,j],fraction=0.05)
  axs[1,j].axis('off')
  axs[1,j].set_title('%s Velocity' % (fig_title[j]))

### Evaluation

In [ ]:
case_num = 3

if(case_num == 1):
  version = 'Z1-4'
  eval_model_name = 'UnetRS_Modelv{}'.format(version)
  eval_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(eval_model_name, eval_model_name ) )

elif(case_num == 2):
  version = 'XYZ1-0'
  eval_model_name = 'UnetRSXYZ_Modelv{}'.format(version)
  eval_model = tf.keras.models.load_model( 'RandomSphere XYZ Model/{}/{}.ckpt'.format(eval_model_name, eval_model_name ) )

elif(case_num == 3):
  version = 'XYZ1-8'
  eval_model_name = 'UnetRSXYZ_Modelv{}'.format(version)
  eval_model = tf.keras.models.load_model( 'RandomSphere XYZ Model/{}/{}.ckpt'.format(eval_model_name, eval_model_name ), custom_objects = {'div_loss2': div_loss2} )

"""
for i in range(sic_solid.shape[0]):
  print('Sample number : {}'.format(i+1))
  eval_model.evaluate(sic_solid[i:i+1,:,:,:,:], sic_vel_minmax[i:i+1,:,:,:,:])
"""

from timeit import default_timer as timer
from datetime import timedelta

start = timer()
vz_test_pred = np.float32( eval_model.predict( x=[sic_solid] ) )
end = timer()
print('Elapsed time : {}'.format(timedelta(seconds=end-start)))

print(vz_test_pred.shape)

In [ ]:
eval_model.save('Saved Model/{}'.format(eval_model_name))

In [ ]:
# Overall Permeability

perm_true = sic_vel_minmax[:,:,:,:,2].mean(axis=(1,2,3))
perm_pred = vz_test_pred[:,:,:,:,2].mean(axis=(1,2,3))

perm_error = abs( (perm_true - perm_pred)/perm_true )*100

print( 'Overall permeability error : {:.3f}\n'.format(perm_error.mean()) )

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(np.arange(perm_error.shape[0]),perm_error)
plt.title('Permeability Error')
plt.show()
"""
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(np.arange(perm_error.shape[0]), vz_test_pred[:,:,:,:].mean(axis=(1,2,3,4)))
plt.title('Average velocity')
plt.show()
"""
print(perm_error)

In [ ]:
# Voxel-wise MAPE Absolute value velocity
v_max = 2
channel = 2

vel_true_flat = sic_vel_minmax[:,:,:,:,channel].flatten()
vel_pred_flat = vz_test_pred[:,:,:,:,channel].flatten()

v_range = np.linspace(0.0001,v_max,100)

vel_true_flat_nonzero = vel_true_flat[ (vel_true_flat < 0) | (vel_true_flat > 0)]
vel_pred_flat_nonzero = vel_pred_flat[ (vel_true_flat < 0) | (vel_true_flat > 0)] 

vel_mape_flat = (vel_true_flat_nonzero - vel_pred_flat_nonzero)/vel_true_flat_nonzero*100

plt.plot(vel_true_flat_nonzero, vel_mape_flat,'bo',markersize=0.1,figure=plt.figure(figsize=[8,6]))
plt.ylim(-100,100)
plt.show()


vel_mape_flat_range = np.zeros_like(v_range)

for i in range(v_range.shape[0]-1):
  if(v_range[i] != 0):
    vel_mape_flat_vrange = np.abs( vel_mape_flat[ (np.abs(vel_true_flat_nonzero) > v_range[i]) & (np.abs(vel_true_flat_nonzero) < v_range[i+1]) ] )

  if(vel_mape_flat_vrange.shape[0] > 0):
    vel_mape_flat_range[i] = np.mean(vel_mape_flat_vrange)

plt.plot(v_range[:-1], vel_mape_flat_range[:-1], 'bo',markersize=4,figure=plt.figure(figsize=[8,6]))
plt.ylim(0,100)
plt.xlabel('velocity')
plt.ylabel('% error')
plt.show()


threshold = 8.93/x_max

vel_mape_flat_threshold = vel_mape_flat[ (vel_true_flat_nonzero <= (-1*threshold)) | (vel_true_flat_nonzero >= (1*threshold)) ]
print("% Error above threshold : {}".format(np.mean(vel_mape_flat_threshold[vel_mape_flat_threshold > 0])))

#print(v_range)
#print(vel_mape_flat_range)

In [ ]:
test_sample = 5
print( 'Maximum Sample Number : {}'.format(sic_solid.shape[0] - 1) )

mySolid = sic_solid[test_sample,:,:,:,0]
mySolid_mask = np.ma.masked_less(mySolid, 1)

myVel_true = sic_vel_minmax[test_sample,:,:,:,2]
myVel_true = np.ma.array( myVel_true, mask=np.ma.getmask(mySolid_mask) )

myVel_pred = vz_test_pred[test_sample,:,:,:,2]
myVel_pred = np.ma.array( myVel_pred, mask=np.ma.getmask(mySolid_mask) )

myVel_true = myVel_true*x_max
myVel_pred = myVel_pred*x_max

test_slice = np.array( [0, 59, -1] )
fig_title = ['Inlet', 'Midpoint', 'Outlet']

fig, axs = plt.subplots( nrows=3, ncols=3,figsize=(20,20) )

vel_range = (0,40)

for j in range(3):

  im=axs[j,0].imshow(mySolid[:,:,test_slice[j]], clim=(0,1), cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[j,0],fraction=0.05)
  axs[j,0].axis('off')
  #axs[j,0].set_title('{} Solid'.format(fig_title[j]))

  im=axs[j,1].imshow(myVel_true[:,:,test_slice[j]], clim=vel_range, cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[j,1],fraction=0.05)
  axs[j,1].axis('off')
  #axs[j,1].set_title('{} Simulation'.format(fig_title[j]))

  im=axs[j,2].imshow(myVel_pred[:,:,test_slice[j]], clim=vel_range, cmap=plt.cm.hot)
  fig.colorbar(im,ax=axs[j,2],fraction=0.05)
  axs[j,2].axis('off')
  #axs[j,2].set_title('{} Prediction'.format(fig_title[j]))

In [ ]:
div = np.zeros( (vz_test_pred.shape[0],118,118,118) )
for i in range(vz_test_pred.shape[0]):

  temp_vel = vz_test_pred[i,:,:,:,:]

  dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
  dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
  dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
  div[i,:,:,:] = dVxdx + dVydy + dVzdz

print( np.mean( np.abs(div), axis=(1,2,3)) )

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.1,0.1,100))
plt.show()

print(np.mean( np.abs(div_flat) ))

In [ ]:
version_x = 'X1-3'
x_model_name = 'UnetRS_Modelv{}'.format(version_x)
x_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(x_model_name, x_model_name ) )
x_model._name = 'xmodel'
x_model.trainable = False

version_y = 'Y1-1'
y_model_name = 'UnetRS_Modelv{}'.format(version_y)
y_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(y_model_name, y_model_name ) )
y_model._name = 'ymodel'
y_model.trainable = False

version_z = 'Z1-4'
z_model_name = 'UnetRS_Modelv{}'.format(version_z)
z_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(z_model_name, z_model_name ) )
z_model._name = 'zmodel'
z_model.trainable = False

vx_test_pred = np.float32( x_model.predict( x=[sic_solid] ) )/2
vy_test_pred = np.float32( y_model.predict( x=[sic_solid] ) )/2
vz_test_pred = np.float32( z_model.predict( x=[sic_solid] ) )

vz_test_pred = np.append(vx_test_pred, np.append(vy_test_pred, vz_test_pred,axis=-1),axis=-1 )
print(vz_test_pred.shape)

In [ ]:
div = np.zeros( (vz_test_pred.shape[0],118,118,118) )
for i in range(vz_test_pred.shape[0]):

  temp_vel = vz_test_pred[i,:,:,:,:]

  dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
  dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
  dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
  div[i,:,:,:] = dVxdx + dVydy + dVzdz

print( np.mean( np.abs(div), axis=(1,2,3)) )

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.1,0.1,100))
plt.show()

### Save prediction

In [ ]:
save_index = 0
case_name = '45PPI_domain1_vfield{}'.format(velocity_dir)

scipy.io.savemat( 'Prediction/{}.mat'.format(case_name), {'vfield_ML': vz_test_pred[save_index,:,:,:]} ) 

In [ ]:
# Individual Model
case_name = (45, 65, 80)

for i in range(6):
  save_name = '{}PPI_domain{}_vfield{}'.format(case_name[i//2], i%2+1, velocity_dir)
  scipy.io.savemat( 'Prediction/UnetRS_Model/{}.mat'.format(save_name), {'vfield_ML': np.squeeze( vz_test_pred[i,:,:,:,:] )} )

In [ ]:
# Comprehensive Model
case_name = (45, 65, 80)
vel_dir_save = ('x', 'y', '')

for i in range(6):
  for j in range(3):
    save_name = '{}PPI_domain{}_vfield{}'.format(case_name[i//2], i%2+1, vel_dir_save[j])
    scipy.io.savemat( 'Prediction/UnetRS_Model/{}.mat'.format(save_name), {'vfield_ML': np.squeeze(vz_test_pred[i,:,:,:,j]) } )

### Divergence check

In [ ]:
sic_solid_temp = sic_solid[1:2,:,:,:,:]
temp_vel = sic_vel_minmax[1,:,:,:,:]

dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
div = dVxdx + dVydy + dVzdz


plt.imshow(sic_solid_temp[0,:,:,59,0])
plt.show()

plt.imshow(temp_vel[:,:,59,2], vmin=0, vmax=6)
plt.colorbar()
plt.show()

plt.imshow(div[:,:,59],vmin=-0.3,vmax=0.3)
plt.colorbar()
plt.show()

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.3,0.3,100))
plt.show()

print(np.mean( np.abs(div_flat) ) )

In [ ]:
sic_solid_temp = sic_solid[1:2,:,:,:,:]

xversion = 'X1-3'
x_model_name = 'UnetRS_Modelv{}'.format(xversion)
x_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(x_model_name, x_model_name ) )

yversion = 'Y1-1'
y_model_name = 'UnetRS_Modelv{}'.format(yversion)
y_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(y_model_name, y_model_name ) )

zversion = 'Z1-4'
z_model_name = 'UnetRS_Modelv{}'.format(zversion)
z_model = tf.keras.models.load_model( 'RandomSphere Model/{}/{}.ckpt'.format(z_model_name, z_model_name ) )

vx_test_pred = np.float32( x_model.predict( x=[sic_solid_temp] ) )/2
vy_test_pred = np.float32( y_model.predict( x=[sic_solid_temp] ) )/2
vz_test_pred = np.float32( z_model.predict( x=[sic_solid_temp] ) )

print(vz_test_pred.shape)

In [ ]:
temp_vel = np.append(vx_test_pred, np.append(vy_test_pred, vz_test_pred,axis=-1),axis=-1 )
print(temp_vel.shape)

temp_vel = temp_vel[0,:,:,:,:]
dVxdx = (temp_vel[2:,1:-1,1:-1,0] - temp_vel[:-2,1:-1,1:-1,0])/2
dVydy = (temp_vel[1:-1,2:,1:-1,1] - temp_vel[1:-1,:-2,1:-1,1])/2
dVzdz = (temp_vel[1:-1,1:-1,2:,2] - temp_vel[1:-1,1:-1,:-2,2])/2
div = dVxdx + dVydy + dVzdz


plt.imshow(sic_solid_temp[0,:,:,59,0])
plt.show()

plt.imshow(temp_vel[:,:,59,2],vmin=0,vmax=6)
plt.colorbar()
plt.show()

plt.imshow(div[:,:,59],vmin=-0.3,vmax=0.3)
plt.colorbar()
plt.show()

div_flat = div.flatten()
plt.hist(div_flat, np.linspace(-0.3,0.3,100))
plt.show()

print(np.mean( np.abs(div_flat) ) )

In [ ]:
temp_div_loss = div_loss1(sic_vel_minmax[1:2,:,:,:,:], np.expand_dims(temp_vel,axis=0))

print(temp_div_loss)